## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir
import pickle

## Compile and program project

In [5]:
project_path_s = [path.curdir, "TrustZone_Flash-S"]
project_path_s

['.', 'TrustZone_Flash-S']

In [6]:
project_path_ns = [path.curdir, "TrustZone_Flash-NS"]
project_path_ns

['.', 'TrustZone_Flash-NS']

In [7]:
atprogram(path.abspath(path.join(*project_path_ns)), verbose=2)
atprogram(path.abspath(path.join(*project_path_s)), verbose=2, erase=False)

make: *** No rule to make target 'all'.  Stop.

make: *** No rule to make target 'all'.  Stop.



2

## Data Logging

In [8]:
live_plot = False

Create a figure for the plot.

In [9]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [10]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_trustzone_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment TrustZone Flash",
}

Stop criteria to pass to the logger:

In [11]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [12]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

In [13]:
print(data)

Interfaces:
	  48:   gpio,    samples:   48011
	 256:   power,   samples: 5811000



# Store Data

In [14]:
import pickle

In [15]:
pickle.dump(data, open("trustzone_flash_logger_data.p", "wb"))

# Load Data

In [16]:
data = pickle.load(open("trustzone_flash_logger_data.p", "rb"))

In [17]:
iteration = 0
name = "TrustZone_Flash"
data = pickle.load(open(path.join(path.pardir, path.pardir, f"{name}_{iteration}.p"), "rb"))

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\TrustZone_Flash_0.p'

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [18]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [19]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [20]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [21]:
print(len(nsc_store_charge), len(nsc_store_time), len(nsc_load_charge), len(nsc_load_time))

6000 6000 6000 6000


In [22]:
drop = 0
cutoff = min(len(nsc_store_charge), len(nsc_store_time), len(nsc_load_charge), len(nsc_load_time)) - drop
nsc_store_charge = nsc_store_charge[:cutoff]
nsc_store_time = nsc_store_time[:cutoff]
nsc_load_charge = nsc_load_charge[:cutoff]
nsc_load_time = nsc_load_time[:cutoff]

# Convert to Joule

In [23]:
voltage = 3.33
j_scale = 1e3 # m
t_scale = 1e3 # m
model_j_scale = 1e6 # n
model_t_scale = 1e3 # u

experiment_name = 'TrustZone'

In [24]:
flash_write_energy = nsc_store_charge[:cutoff]
flash_read_energy = nsc_load_charge[:cutoff]
flash_write_time_s = nsc_store_time[:cutoff]
flash_read_time_s = nsc_load_time[:cutoff]

for i in range(len(flash_write_energy)):
    flash_write_energy[i] = flash_write_energy[i] * voltage * j_scale
for i in range(len(flash_read_energy)):
    flash_read_energy[i] = flash_read_energy[i] * voltage * j_scale
    
for i in range(len(flash_write_time_s)):
    flash_write_time_s[i] = flash_write_time_s[i] * t_scale
for i in range(len(flash_read_time_s)):
    flash_read_time_s[i] = flash_read_time_s[i] * t_scale

In [25]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(flash_write_energy))
print(f"MAX_NUM_BYTES: {num_bytes[-1]}")

MAX_NUM_BYTES: 6000


In [26]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)
# pars['intercept'].set(min=0)

In [27]:
results = []
ylabels = ['Energy [mJ]'] * 2 + ['Time [ms]'] * 2 + ['Energy [mJ]'] + ['Time [ms]']
parameter_names = [
    'Flash Write Energy',
    'Flash Read Energy',
    'Flash Write Time',
    'Flash Read Time',
    'Total Energy',
    'Total Time',
]
for y in [flash_write_energy, flash_read_energy, flash_write_time_s, flash_read_time_s, 
          [w + r for (w,r) in zip(flash_write_energy, flash_read_energy)],
          [w + r for (w,r) in zip(flash_write_time_s, flash_read_time_s)]]:
#     result = mod.fit(y[16:-16], pars, x=num_bytes[16:-16])
    result = mod.fit(y, pars, x=num_bytes) 
    print(result.fit_report())
    fig, grid = result.plot(
        xlabel='Checkpoint Size [Bytes]',
        ylabel=ylabels[len(results)])
    fig.tight_layout(rect=(0.05, 0.05, 1, 1))
    fig.set_size_inches(5, 4.5, forward=True)
    fig.canvas.set_window_title(
        f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 6000
    # variables        = 2
    chi-square         = 0.13893338
    reduced chi-square = 2.3163e-05
    Akaike info crit   = -64035.6531
    Bayesian info crit = -64022.2541
[[Variables]]
    slope:      8.2290e-05 +/- 3.5873e-08 (0.04%) (init = 0)
    intercept:  0.01389407 +/- 1.2428e-04 (0.89%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 6000
    # variables        = 2
    chi-square         = 2.3081e-04
    reduced chi-square = 3.8481e-08
    Akaike info crit   = -102436.628
    Bayesian info crit = -102423.229
[[Variables]]
    slope:      2.4787e-06 +/- 1.4621e-09 (0.06%) (init = 0)
    intercept: -7.2067e-06 +/- 5.0656e-06 (70.29%) (init = 1)
[[Correlations]] (unrep

In [28]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title(f"Analysis {experiment_name}")

In [29]:
charge_color = 'r'
time_color = 'b'

In [30]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [mJ]', color=charge_color)
ax2.set_ylabel('Time [ms]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [31]:
lines = []
lines += ax1.plot(num_bytes, flash_write_energy, charge_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, flash_read_energy, charge_color+'--', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, flash_write_time_s, time_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, flash_read_time_s, time_color+'--', label=f'{parameter_names[len(lines)]}')
ax1.legend(handles=lines)
# [flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[2].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[3].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[4].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.04} $\mu$s\n")
fig2.tight_layout()
fig2.show()

In [32]:
print(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[2].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[3].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[4].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.020} $\mu$s\n"
)

Flash Write Energy: Slope 82.289845195767142627 nJ/B, Intercept 13894.072862542294388 nJ
Flash Read Energy: Slope 2.4786872203547027915 nJ/B, Intercept -7.2066974816653583957 nJ
Flash Write Time: Slope 12.790787139796535143 $\mu$s/B, Intercept 2084.8312465931671795 $\mu$s
Flash Read Time: Slope 2.1203929853301453079 $\mu$s/B, Intercept 18.491442017488868288 $\mu$s
Total Energy: Slope 84.768532416120663697 nJ/B, Intercept 13886.866167242773372 nJ
Total Time: Slope 14.911180125126723084 $\mu$s/B, Intercept 2103.3226886128072692 $\mu$s



In [33]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(flash_write_energy, open("trustzone_flash_write_energy_mJ.p", "wb"))
pickle.dump(flash_read_energy, open("trustzone_flash_read_energy_mJ.p", "wb"))
pickle.dump(flash_write_time_s, open("trustzone_flash_write_time_ms.p", "wb"))
pickle.dump(flash_read_time_s, open("trustzone_flash_read_time_ms.p", "wb"))

## Write config file

In [34]:
import json

config = {}
config["name"] = "TrustZone"
config["project_paths"] = [project_path_ns, project_path_s]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["TrustZone Flash Write"], 3: ["TrustZone Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["TrustZone Flash Read"],
                                        "load": ["TrustZone Flash Write"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": 1}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

# Write model data

In [35]:
dump_pickle = True
fit_lm = True
verbose = 2
show_lm_plot = 2

In [36]:
# Parse data
analysis_config = config.get("analysis")
result_types = analysis_config.get("result_types")
x_step = analysis_config.get("x_step")
parsed_data = {}
for pin, parameter_names in analysis_config.get("pins").items():
    data2 = power_and_time_per_pulse(
        data, int(pin), stop_function=stop_function)
    num_names = len(parameter_names)
    for i, parameter_name in enumerate(parameter_names):
        end_index = -drop * num_names or None
        parsed_data[parameter_name] = {
            result_types[0]: data2[0][i:end_index:num_names],
            result_types[1]: data2[1][i:end_index:num_names],
            "x_step": x_step}
if dump_pickle:
    pickle.dump(parsed_data, open(
        path.join(path.curdir,
                  f"{config_dict.get('file_name_base')}_looped.p"), "wb"))

In [37]:
# Fit lm
if fit_lm:
    model = None
    if model is None:
        def line(x, intercept, slope):
            """a line"""
            return [intercept + slope*i for i in x]

        model = Model(line)
        params = model.make_params(intercept=0, slope=1)
#         params['intercept'].set(min=0)
    else:
        params = model.params

    model_results = {}
    labels = analysis_config.get("labels")
    for parameter_name in parsed_data.keys():
        length = len(parsed_data[parameter_name][result_types[0]])
        x_step = parsed_data[parameter_name]["x_step"]
        num_bytes = range(x_step, (length+1)*x_step, x_step)
        if verbose:
            print(
                f"Fitting model to {parameter_name} with {length} " +
                f"samples, from {min(num_bytes)} to {max(num_bytes)} "
                f"bytes in steps of {x_step}.")
        model_result = {}
        for result_type in result_types:
            model_result[result_type] = model.fit(
                parsed_data[parameter_name][result_type], params,
                x=num_bytes)
            if verbose >= 2:
                print(model_result[result_type].fit_report())
            # Plot multiple view
            if show_lm_plot >= 2:
                fig, grid = model_result[result_type].plot(
                    xlabel=f"{labels[result_type]['x']} " +
                           f"[{labels[result_type]['x_unit']}]",
                    ylabel=f"{labels[result_type]['y']} " +
                           f"[{labels[result_type]['y_unit']}]")
                fig.canvas.set_window_title(
                    f"Residuals of {parameter_name}")
                fig.tight_layout()
                fig.show()
            model_results[parameter_name] = model_result

    # Plot single view
    if show_lm_plot:
        import matplotlib.pyplot as plt
        fig = plt.figure(figsize=(9, 6))
        fig.canvas.set_window_title(f"Analysis {config.get('name')}")
        colors = dict(zip(result_types, ['r', 'b']))
        line_styles = (
            line_style for line_style in ('-', '--', '-.', ':') * 2)
        # fig.suptitle(f"Energy analysis of {config.get('name')}")
        ax = {}
        ax[result_types[0]] = fig.add_subplot(1, 1, 1)
        ax[result_types[1]] = ax[result_types[0]].twinx()
        ax[result_types[0]].set_xlabel(
            f"{labels[result_types[0]]['x']} " +
            f"[{labels[result_types[0]]['x_unit']}]")
        for result_type in result_types:
            ax[result_type].set_ylabel(
                f"{labels[result_type]['y']} " +
                f"[{labels[result_type]['y_unit']}]",
                color=colors[result_type])
            ax[result_type].tick_params('y', colors=colors[result_type])

        lines = []
        title_str = ""
        for parameter_name in parsed_data.keys():
            length = len(parsed_data[parameter_name][result_types[0]])
            x_step = parsed_data[parameter_name]["x_step"]
            num_bytes = range(x_step, (length+1)*x_step, x_step)
            model_result = {}
            line_style = next(line_styles)
            for result_type in result_types:
                label = f"{parameter_name} {labels[result_type]['y']}"
                lines += ax[result_type].plot(
                    num_bytes, parsed_data[parameter_name][result_type],
                    colors[result_type] + line_style, label=label)
                title_str += f"{label} "
                for param in params.keys():
                    title_str += "".join(
                        f"{params[param].name.capitalize()}: ")
                    title_str += "".join(
                        f"{model_results[parameter_name][result_type].params[param].value: .03} ")
                    title_str += "".join(
                        f"{labels[result_type]['y_unit']}, ")
                title_str = title_str[:-2] + \
                    f" per {labels[result_type]['x_unit']}\n"
        ax[result_types[0]].legend(handles=lines)
        ax[result_types[0]].set_title(title_str[:-1])
        # fig.tight_layout()
        fig.tight_layout(rect=(0.05, 0.05, 1, 1))
        fig.set_size_inches(8, 6, forward=True)
        fig.show()

    # Save model results to file
    if dump_pickle:
        model_results_dump = {}
        for parameter_name in model_results.keys():
            model_results_dump[parameter_name] = {}
            for result_type in model_results[parameter_name].keys():
                model_results_dump[parameter_name][result_type] = \
                    model_results[parameter_name][result_type].values

        pickle.dump(model_results_dump, open(path.join(
            path.curdir,
            f"{config_dict.get('file_name_base')}_model.p"), "wb"))

Fitting model to TrustZone Flash Write with 6000 samples, from 1 to 6000 bytes in steps of 1.
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 9
    # data points      = 6000
    # variables        = 2
    chi-square         = 1.2529e-08
    reduced chi-square = 2.0889e-12
    Akaike info crit   = -161364.384
    Bayesian info crit = -161350.985
[[Variables]]
    intercept:  4.1724e-06 +/- 3.7322e-08 (0.89%) (init = 0)
    slope:      2.4712e-08 +/- 1.0773e-11 (0.04%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(intercept, slope) = -0.866
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 6000
    # variables        = 2
    chi-square         = 0.00270732
    reduced chi-square = 4.5137e-07
    Akaike info crit   = -87663.8703
    Bayesian info crit = -87650.4712
[[Variables]]
    intercept:  0.00208483 +/- 1.7349e-05 (0.83%) (i